# Financial Market Intelligence RAG System
## CS6120 Final Project - Colab Version

**Team Members:** Soonbee Hwang & Xinyuan Fan (Amber)

This notebook runs the complete RAG system in Google Colab.

**Note:** Enable GPU runtime (Runtime → Change runtime type → GPU) for faster processing.


## Step 1: Install Dependencies


In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected! Consider enabling GPU runtime for better performance.")
    print("   Go to: Runtime → Change runtime type → GPU")

# Install all required packages
!pip install -q torch transformers sentence-transformers faiss-cpu numpy pandas beautifulsoup4 lxml tiktoken streamlit pyngrok

# Install llama-cpp-python (supports both CPU and GPU)
# For GPU support, we'll use the default installation which auto-detects CUDA
!pip install -q llama-cpp-python

# Install Kaggle API
!pip install -q kaggle

print("✅ All dependencies installed!")


## Step 2: Download Dataset

### Using Kaggle API Token

1. Get your API token from https://www.kaggle.com/settings
2. Click "Create New Token" and copy the token (format: `KGAT_xxxxx`)
3. Update the code below with your token and username


In [ ]:
# Using API Token (No file upload needed)
import os

KAGGLE_API_TOKEN = "KGAT_78cd36a5f2ad172b13fe785d6976fdee"  # Replace with your token

# Set environment variable
os.environ['KAGGLE_API_TOKEN'] = KAGGLE_API_TOKEN

# Also set username (get it from https://www.kaggle.com/settings)
KAGGLE_USERNAME = "amberfxy"  # Your Kaggle username
os.environ['KAGGLE_USERNAME'] = KAGGLE_USERNAME

print("✅ Kaggle API token configured!")
print(f"   Username: {KAGGLE_USERNAME}")


## Step 3: Download LLM Model

Download Mistral 7B GGUF model (~4.1GB).


In [ ]:
# Download dataset
import os

os.makedirs('data/raw', exist_ok=True)

# Download the dataset
!cd data/raw && kaggle datasets download -d aaron7sun/stocknews

# Unzip the dataset
!cd data/raw && unzip -q stocknews.zip

print("✅ Dataset downloaded!")


## Step 4: Clone Project Code

Clone the project repository from GitHub.


In [ ]:
import os
os.makedirs('models', exist_ok=True)

# Download Mistral 7B model
!cd models && wget -q --show-progress https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf

print("✅ Model downloaded!")


In [ ]:
# Clone from GitHub
!git clone https://github.com/amberfxy/financial-market-intelligence-rag.git
!cp -r financial-market-intelligence-rag/src .
!cp -r financial-market-intelligence-rag/ui .
!cp -r financial-market-intelligence-rag/scripts .

print("✅ Code downloaded!")


## Step 5: Build FAISS Index

Load data, generate embeddings, and build the vector store.


In [ ]:
import sys
sys.path.insert(0, '.')

import pandas as pd
import logging
import torch
from src.data.loader import load_kaggle_dataset, preprocess_data
from src.chunking.chunker import chunk_dataframe
from src.embeddings.embedder import BGEEmbedder
from src.vectorstore.faiss_store import FAISSStore

logging.basicConfig(level=logging.INFO)

# Load and preprocess data
print("Loading dataset...")
df = load_kaggle_dataset('data/raw')
df = preprocess_data(df)

# Chunk documents
print("Chunking documents...")
chunks = chunk_dataframe(df, text_column='News Headline', max_tokens=250)

# Generate embeddings
print("Generating embeddings...")
# BGE model will automatically use GPU if available
embedder = BGEEmbedder()
chunk_texts = [chunk["text"] for chunk in chunks]

# Use larger batch size on GPU for faster processing
batch_size = 128 if torch.cuda.is_available() else 32
print(f"Using batch size: {batch_size} ({'GPU' if torch.cuda.is_available() else 'CPU'})")

embeddings = embedder.embed_texts(chunk_texts, batch_size=batch_size)

# Build FAISS index
print("Building FAISS index...")
os.makedirs('vectorstore', exist_ok=True)
vectorstore = FAISSStore(dimension=embeddings.shape[1])
vectorstore.add_chunks(embeddings, chunks)
vectorstore.save('vectorstore/faiss.index', 'vectorstore/chunks.pkl')

print(f"✅ Index built successfully! Total chunks: {len(chunks)}")


## Step 6: Run Streamlit Application

Start the Streamlit web interface for querying the RAG system.


In [ ]:
# Note: Use Colab's built-in port forwarding to access the Streamlit app
# Click on the URL shown after running the next cell


In [ ]:
# Run Streamlit app
!streamlit run ui/app.py --server.port=8501 --server.address=0.0.0.0


## Notes

- **GPU Runtime**: Enable GPU (Runtime → Change runtime type → GPU) for faster processing
- **Data Persistence**: Colab sessions reset when disconnected. Save important files to Google Drive if needed
- **Session Timeout**: Colab sessions timeout after 90 minutes of inactivity
